In [1]:
print('hello. this notebook lists all files in files folder, process with easy ocr, detect names and append it into excel file excel.xlsx, ')

hello. this notebook lists all files in files folder, process with easy ocr, detect names and append it into excel file excel.xlsx, 


In [2]:
import easyocr
import os
import cv2

# Create an EasyOCR reader
reader = easyocr.Reader(['en'])  # Specify the languages you need, e.g., 'en' for English

# Path to the directory containing your image files
folder_path = './files'

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    print(filename)
    if filename.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        # Path to the image file
        img_path = os.path.join(folder_path, filename)
        print(img_path)
        # Read the image using OpenCV
        img = cv2.imread(img_path)

        if img is None:
            print(f"Error: The image file {img_path} is empty or not found.")
            continue  # Skip to the next file if the image is empty

        # Resize the image (e.g., resize to 2000 pixels on the longest side)
        max_dimension = 2000
        height, width = img.shape[:2]
        if max(height, width) > max_dimension:
            scaling_factor = max_dimension / float(max(height, width))
            new_size = (int(width * scaling_factor), int(height * scaling_factor))
            img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)
            cv2.imwrite(img_path, img)

        # Use EasyOCR to extract text from the image
        result = reader.readtext(img_path)
        text = ' '.join([res[1] for res in result])
        
        # Print the extracted text
        print(f'Text from {filename}:\n{text}\n')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


20241226_013233.jpg
./files\20241226_013233.jpg
Text from 20241226_013233.jpg:
HR Ponovite korake dok Vam ne ponestane Vrpce: EN Repeat step &-d untll you have Just a Ilttle thick ramalnlng; Ismetelje a2 &)-d) lepeseket & zslnor vegelg: MZ Blr U2 qalin Ip qulana qodor &-d addimlarni tokrarloyin; FU Kipetey a tabadtno esaurire la lunghezza del cordoncina DG nlobropere crbnkare Or "" Ao %r" AO MjvepnBaNe HA AbnxkMara Ha LT Pakartokite $-d IIngsnlus; kol balgsls vlrvele; opbbta LV Atkartot darbibas &-d Iidz beldzas auklas garums DIH Ponovite 4 d dok Vam ne ponestane uzice PL Powtorzyk krokl 4-da2 do momentu wyczerpanla dlugolcl Opakulte kroky al do konce delky provaU: wnurka, DE Wlederholen Sle dle Schrhte # d; blydie LAnge der Kordel erschoph IJL PT Repetir 0S Dassos &-d a1d sa esgolar 0 comprimento do cordel: Koriake samme d,kuni nobr plkkus 4440 0tya. RO Repatafil pajlla-d pJnd (Andva rJmdne doar pufin cordon Replta Ios pasos d haste egotar Ia longitud del cable As Ponovlte karake &-d 

In [3]:
import pandas as pd

# Path to your Excel file
excel_file = './excel.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(excel_file)

# Display the DataFrame
print(df)


Empty DataFrame
Columns: [Entry, Name, Filename]
Index: []


In [4]:
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import SpacyNlpEngine

# Initialize the NLP engine
nlp_engine = SpacyNlpEngine()

# Create the analyzer engine
analyzer = AnalyzerEngine(nlp_engine=nlp_engine)

# Define the text to analyze
text = """ {
    "en": "Sophy Santino and John Doe are meeting tomorrow.",
    "de": "Sophy Santino und John Doe treffen sich morgen.",
    "pl": "Sophy Santino i John Doe spotkają się jutro.",
    "sk": "Frantisek Karovany a Peter Repa trhali travu od rana.",
    "cz": "Jiří Kočí ráno z okna skočí."
}"""

# Analyze the text for PII entities
results = analyzer.analyze(text=text, entities=["PERSON"], language="en")

# Define a confidence threshold
confidence_threshold = 0.85

# Print the detected names
for result in results:
    if result.entity_type == "PERSON":
        if result.entity_type == "PERSON" and result.score >= confidence_threshold:
            detected_name = text[result.start:result.end]
            print(f"Detected name: {detected_name} (confidence: {result.score})")


#add more languages:
# terminal: python -m spacy download de_core_news_sm



Detected name: Sophy Santino (confidence: 0.85)
Detected name: John Doe (confidence: 0.85)
Detected name: Sophy Santino (confidence: 0.85)
Detected name: John Doe (confidence: 0.85)
Detected name: sich morgen (confidence: 0.85)
Detected name: Sophy Santino (confidence: 0.85)
Detected name: John Doe (confidence: 0.85)
Detected name: się jutro (confidence: 0.85)
Detected name: Frantisek Karovany (confidence: 0.85)
Detected name: Peter Repa (confidence: 0.85)
Detected name: od rana (confidence: 0.85)
Detected name: Jiří Kočí (confidence: 0.85)


In [5]:
for result in results:
    if result.entity_type == "PERSON":
        detected_name = text[result.start:result.end]

        #add a condition if the name is known

        # Get the current number of rows 
        new_row_index = df.shape[0] 
        df.loc[new_row_index] = [new_row_index +1, detected_name, filename] # Integrate with listdir loop in 1st cell
        print(f"Detected name: {detected_name}")

print(df.shape[0])
df.head()

Detected name: Sophy Santino
Detected name: John Doe
Detected name: Sophy Santino
Detected name: John Doe
Detected name: sich morgen
Detected name: Sophy Santino
Detected name: John Doe
Detected name: się jutro
Detected name: Frantisek Karovany
Detected name: Peter Repa
Detected name: od rana
Detected name: Jiří Kočí
12


,Entry,Name,Filename
0,1,Sophy Santino,20241226_115344.jpg
1,2,John Doe,20241226_115344.jpg
2,3,Sophy Santino,20241226_115344.jpg
3,4,John Doe,20241226_115344.jpg
4,5,sich morgen,20241226_115344.jpg


In [6]:
# from transformers import pipeline

# # Load a pre-trained model for NER
# ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# def is_name(string):
#     entities = ner(string)
#     for entity in entities:
#         if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
#             print('True: ', string)
#             return True
#     print('False: ', string)
#     return False

# # Create a new DataFrame to store valid names
# valid_df = pd.DataFrame(columns=df.columns)

# # Iterate through each row in the DataFrame
# for index, row in df.iterrows():
#     if is_name(row['Name']):
#         valid_df = valid_df.append(row)

# # Reset index of the new DataFrame
# valid_df.reset_index(drop=True, inplace=True)

# # Print the filtered DataFrame
# print(valid_df)




In [10]:
import pandas as pd
from transformers import pipeline

# Load a pre-trained model for NER
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

def is_name(string):
    entities = ner(string)
    for entity in entities:
        if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
            print('True: ', string)
            return True
    print('False: ', string)
    return False


# Filter rows where 'Name' is a valid name
valid_df = df[df['Name'].apply(is_name)]

# Reset index of the filtered DataFrame
valid_df.reset_index(drop=True, inplace=True)

for i in range(0, valid_df.shape[0]):
    valid_df.loc[i, 'Entry'] = i+1

# Print the filtered DataFrame
print(valid_df)


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


True:  Sophy Santino
True:  John Doe
True:  Sophy Santino
True:  John Doe
True:  Sophy Santino
True:  John Doe
True:  Frantisek Karovany
True:  Peter Repa
True:  Jiří Kočí
   Entry                Name             Filename
0      1       Sophy Santino  20241226_115344.jpg
1      2            John Doe  20241226_115344.jpg
2      3       Sophy Santino  20241226_115344.jpg
3      4            John Doe  20241226_115344.jpg
4      5       Sophy Santino  20241226_115344.jpg
5      6            John Doe  20241226_115344.jpg
6      7  Frantisek Karovany  20241226_115344.jpg
7      8          Peter Repa  20241226_115344.jpg
8      9           Jiří Kočí  20241226_115344.jpg


In [8]:
df = valid_df

df.to_excel('excel.xlsx', index=False)
df.head(15)

,Entry,Name,Filename
0,1,Sophy Santino,20241226_115344.jpg
1,2,John Doe,20241226_115344.jpg
2,3,Sophy Santino,20241226_115344.jpg
3,4,John Doe,20241226_115344.jpg
4,6,Sophy Santino,20241226_115344.jpg
5,7,John Doe,20241226_115344.jpg
6,9,Frantisek Karovany,20241226_115344.jpg
7,10,Peter Repa,20241226_115344.jpg
8,12,Jiří Kočí,20241226_115344.jpg


## Extensions: new PII languages ##
